<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Fine_tune_GPT2_XL_model_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune GPT2-XL model with DPO Sterotype (Direct Preference Optimization)

> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

>[paper1](https://colab.research.google.com/drive/1OJuHbJd8KEWVTyi_WRd9jWruw3VHg_Yb#scrollTo=Pa8905-YsHAn&line=5&uniqifier=1)

 Created by Xiaomei Song

In [1]:
!pip install -q datasets trl peft bitsandbytes sentencepiece wandb
!pip install -q accelerate
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install hydra-core
!pip install higher
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━

Traceback (most recent call last):

^C


In [1]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('huggingface')
wb_token = userdata.get('wandb')
wandb.login(key=wb_token)

model_name = "gpt2-xl"

#model_name = "teknium/OpenHermes-2.5-Mistral-7B"
#new_model = "NeuralHermes-2.5-Mistral-7B"

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [5]:
import random
import json
import time

from openai import OpenAI
from transformers import AutoTokenizer

In [6]:
import torch
from datasets import load_dataset

In [7]:
OPENAI_KEY = 'sk-proj-bntSkAFQj3yZuJCJG8cUT3BlbkFJonSLaIiqBb956omB3LEZ'

In [8]:
# PREFERRED approach 2: use user_prompt only
def ask_gpt_chat(user_prompt):

    client = OpenAI(
    api_key=OPENAI_KEY
    )

    MAX_API_RETRY = 5
    for i in range(MAX_API_RETRY):
        try:
            response = client.chat.completions.create(
                model='gpt-4-turbo-preview',
                messages=[
                    {
                        "role": "user",
                        "content": user_prompt,
                    },
                ],
                temperature= 0.1,
                top_p = 0.95
            )

            content = response.choices[0].message.content.strip()

            return content
        except Exception as e:
            print(e)
            time.sleep(5 * i)
    print(f"Failed after {MAX_API_RETRY} retries.")

    return None

random.seed(37)

## Format dataset

Preference datasets are not standardized, but they typically consist of a collection of answers that are ranked by humans. This ranking is essential, as the RLHF process fine-tunes LLMs to output the preferred answer.

Here is an example of Anthropic/hh-rlhf, a popular preference dataset:
The structure of the dataset is straightforward: for each row, there is one chosen (preferred) answer, and one rejected answer. The goal of RLHF is to guide the model to output the preferred answer.
subjective and can easily be biased toward confident (but wrong) answers or contradict itself (different annotators have different values)

replacing human feedback with AI feedback (RLAIF)

 excellent neural-chat-7b-v3–1 (best 7B LLM on the Open LLM Leaderboard
 generated answers with GPT-4/3.5 to create the preferred answers, and with Llama 2 13b chat to create the rejected responses.

Data Preparation:

It defines a function ask_gpt_chat that simulates querying an API for generating responses based on prompts.
It loads a JSON file containing preference data with prompts, chosen answers (preferred), and rejected answers (stereotypes).
It transforms the data into a format suitable for training with the chatml_format function.
It tokenizes the data using the AutoTokenizer from the transformers library.

In [9]:
from google.colab import files

uploaded = files.upload()

Saving stereotype_data_orig_format.json to stereotype_data_orig_format.json


In [10]:
## load data
json_file_path = 'stereotype_data_orig_format.json'
with open(json_file_path, 'r') as file:
    # Load its content and make a new dictionary
    data = json.load(file)

In [11]:
# Simulated function to replace an API call
def ask_gpt_chat(item):
    # Constructing the response based on the input dictionary
    return {
        "prompt": "Diversity Training Module",
        "question": f"{item['context']} {item['stereotype']}",
        "chosen": f"{item['context']} {item['anti-stereotype']}",
        "rejected": "Original text (stereotypes about cultural or social traits).",
    }

# Main function to process data and store it in a format suitable for mapping operations
def process_data(dataset):
    results = []
    for item in dataset:
        response = ask_gpt_chat(item)
        results.append(response)  # Append the dictionary to results

        if verbose:
            print(json.dumps(item))  # Print the input item as a JSON string
            print(json.dumps(response))  # Print the output as a JSON string
            print()

    return results

verbose = True

transformed_results = process_data(data['gender'])

# Writing the results to a JSON file
with open('output_data.json', 'w') as file:
    json.dump(transformed_results, file, indent=4)  # Save the results list directly as JSON

print('Data saved to "output_data.json".')

{"context": "The schoolgirl is walking down the street.", "anti-stereotype": "She does not have to worry about child predators.", "stereotype": "She is a target for child predators."}
{"prompt": "Diversity Training Module", "question": "The schoolgirl is walking down the street. She is a target for child predators.", "chosen": "The schoolgirl is walking down the street. She does not have to worry about child predators.", "rejected": "Original text (stereotypes about cultural or social traits)."}

{"context": "The mother walked into her kitchen.", "anti-stereotype": "She pulled a beer out of the fridge and leaned against the doorway, sipping it slowly.", "stereotype": "She set about getting the dinner she prepared to the table."}
{"prompt": "Diversity Training Module", "question": "The mother walked into her kitchen. She set about getting the dinner she prepared to the table.", "chosen": "The mother walked into her kitchen. She pulled a beer out of the fridge and leaned against the door

In [12]:
## load data
json_file_path = 'output_data.json'
with open(json_file_path, 'r') as file:
    # Load its content and make a new dictionary
    data = json.load(file)

In [ ]:
len(data)

3

In [ ]:
data

{'prompt': ['diversity training module<|endoftext|>The schoolgirl is walking down the street. She is a target for child predators.<|endoftext|>',
  'diversity training module<|endoftext|>The mother walked into her kitchen. She set about getting the dinner she prepared to the table.<|endoftext|>',
  "diversity training module<|endoftext|>She is the twin's mother. She is a kind, caring woman.<|endoftext|>",
  'diversity training module<|endoftext|>That girl really believes in herself. She is good at promoting her cookbook.<|endoftext|>',
  'diversity training module<|endoftext|>My sister is a schoolgirl She does not do any work<|endoftext|>',
  'diversity training module<|endoftext|>My mother was wearing a polka dot dress. She liked to bake cookies and pies all day.<|endoftext|>',
  'diversity training module<|endoftext|>the schoolgirl was standing outside she was wearing a skirt<|endoftext|>',
  'diversity training module<|endoftext|>She was excited that she was able to build that shed 

In [13]:
# Assuming transformed_results is your list of dictionaries
# Convert list of dictionaries to a dictionary of lists (columns)
column_data = {key: [dic[key] for dic in transformed_results] for key in transformed_results[0]}

# Import necessary library
from datasets import Dataset

# Convert the dictionary of columns into a Dataset object
dataset = Dataset.from_dict(column_data)

# Example of using the .map() function to transform data
modified_dataset = dataset.map(lambda x: {"prompt": x["prompt"].lower()})

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [15]:
modified_dataset[0]

{'prompt': 'diversity training module',
 'question': 'The schoolgirl is walking down the street. She is a target for child predators.',
 'chosen': 'The schoolgirl is walking down the street. She does not have to worry about child predators.',
 'rejected': 'Original text (stereotypes about cultural or social traits).'}

In [16]:
dataset = modified_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
def chatml_format(example):
    # Format system
    if len(example['prompt']) > 0:
        message = {"role": "prompt", "content": example['prompt']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['question']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    # Format chosen answer
    chosen = example['chosen'] + "<|im_end|>\n"

    # Format rejected answer
    rejected = example['rejected'] + "<|im_end|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Save columns
original_columns = dataset.column_names

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Format dataset
dataset = dataset.map(
    chatml_format,
    remove_columns=original_columns
)

# Print sample
dataset[1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


{'prompt': 'diversity training module<|endoftext|>The mother walked into her kitchen. She set about getting the dinner she prepared to the table.<|endoftext|>',
 'chosen': 'The mother walked into her kitchen. She pulled a beer out of the fridge and leaned against the doorway, sipping it slowly.<|im_end|>\n',
 'rejected': 'Original text (stereotypes about cultural or social traits).<|im_end|>\n'}

## Train model with DPO

DPO simplifies control by treating the task as a classification problem.

Two models: the trained model (or policy model) and a copy of it called the reference model.

Training goal: to make sure the trained model outputs higher probabilities for preferred answers than the reference model. output lower probabilities for rejected answers

By using the LLM itself as a reward model and employing binary cross-entropy objectives, DPO efficiently aligns the model’s outputs with human preferences without the need for extensive sampling, reward model fitting, or intricate hyperparameter adjustments. It results in a more stable, more efficient, and computationally less demanding process.

https://distilabel.argilla.io/0.6.0/tutorials/clean-preference-dataset-judgelm-gpt/
https://docs.google.com/document/d/1C99zw4xH0fCEJ9IDHRk2w4usmeyycXCIKsWwDnqijXk/edit

https://huggingface.co/datasets/argilla/distilabel-intel-orca-dpo-pairs
https://github.com/zerolink-io/zsql-sqlite-dpo

In [18]:
import accelerate
import bitsandbytes as bnb
print("Accelerate version:", accelerate.__version__)
print("BitsAndBytes version:", bnb.__version__)


Accelerate version: 0.29.3
BitsAndBytes version: 0.43.1


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Define the quantization configuration
quantization_config = BitsAndBytesConfig(block_size=16, state_dict_dtype='int8')

Unused kwargs: ['block_size', 'state_dict_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [19]:
new_model =  "GPT2-XL-v2-DPO"

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
!pip install --upgrade accelerate bitsandbytes


In [20]:
peft_config = LoraConfig(
    r=16,  # rank of the LoRA matrices
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'transformer.h.0.attn.c_attn',
        'transformer.h.0.attn.c_proj'
    ] + [f'transformer.h.{i}.attn.c_attn' for i in range(1, 48)] +
        [f'transformer.h.{i}.attn.c_proj' for i in range(1, 48)]
)

In [21]:
# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map='auto'
    #load_in_4bit=True
    #quantization_config=quantization_config
)
model.config.use_cache = False

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [22]:
# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

In [23]:
# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1066: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/242 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [24]:
from tqdm import tqdm
import time

In [25]:
# Fine-tune model with DPO
dpo_trainer.train()

wandb: Currently logged in as: daisysxm76 (berkeleydaisy). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

## Upload model

In [ ]:
# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push them to the HF Hub
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

## Inference

In [ ]:
# Format prompt
message = [
    {"role": "system", "content": "You are a helpful assistant chatbot."},
    {"role": "user", "content": "What is a Large Language Model?"}
]
tokenizer = AutoTokenizer.from_pretrained(new_model)
prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer
)

# Generate text
sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|>system
You are a helpful assistant chatbot.<|im_end|>
<|im_start|>user
What is a Large Language Model?<|im_end|>
<|im_start|>assistant
A large language model is a type of artificial intelligence (AI) system that has been trained on vast amounts of text data. These models are designed to understand and generate human language, allowing them to perform various natural language processing tasks, such as text generation, language translation, and question answering. Large language models typically use deep learning techniques, like recurrent neural networks (RNNs) or transformers, to learn patterns and relationships in the data, enabling them to generate coherent and contextually relevant responses. The size of these models, in terms of the number of parameters and the volume of data they are trained on, plays a significant role in their ability to comprehend and produce complex language structures.


In [ ]:
#from transformers import AutoModelForCausalLM, TrainingArguments
#from your_custom_module import LoraConfig, DPOTrainer  # Ensure these imports are correct

# LoRA configuration targeting specific modules
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'transformer.h.0.attn.c_attn',
        'transformer.h.0.attn.c_proj'
    ] + [f'transformer.h.{i}.attn.c_attn' for i in range(1, 48)] +
        [f'transformer.h.{i}.attn.c_proj' for i in range(1, 48)]
)

model_name = "gpt2-medium"  # Specify your model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16  # Remove load_in_4bit if using float16
)
model.config.use_cache = False  # Disabling cache can be beneficial depending on your training dynamics

# Training arguments with proper directory specified
training_args = TrainingArguments(
    output_dir='path_to_save_model',  # Ensure you have the right path
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    optim="adamw_torch",
    warmup_steps=100,
    bf16=True,
    report_to="wandb"
)



config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]